In [22]:
import datetime

def parse_first_brewed(text: str) -> datetime.date:
    parts = text.split('/')
    if len(parts) == 2:
        return datetime.date(int(parts[1]), int(parts[0]), 1)
    elif len(parts) == 1:
        return datetime.date(int(parts[0]), 1, 1)
    else:
        assert False, 'Unknown date format'
        

       

In [2]:
>>> parse_first_brewed('09/2007')

datetime.date(2007, 9, 1)

NameError: name 'beer' is not defined

In [4]:
from typing import Iterator, Dict, Any
from urllib.parse import urlencode
import requests

def iter_beers_from_api(page_size: int = 5) -> Iterator[Dict[str, Any]]:
    session = requests.Session()
    page = 1
    while True:
        response = session.get('https://api.punkapi.com/v2/beers?' + urlencode({
            'page': page,
            'per_page': page_size
        }))
        response.raise_for_status()

        data = response.json()
        if not data:
            break

        yield from data

        page += 1

In [25]:
beers = iter_beers_from_api()


NameError: name 'beer' is not defined

In [6]:
>>> next(beers)

{'id': 1,
 'name': 'Buzz',
 'tagline': 'A Real Bitter Experience.',
 'first_brewed': '09/2007',
 'description': 'A light, crisp and bitter IPA brewed with English and American hops. A small batch brewed only once.',
 'image_url': 'https://images.punkapi.com/v2/keg.png',
 'abv': 4.5,
 'ibu': 60,
 'target_fg': 1010,
 'target_og': 1044,
 'ebc': 20,
 'srm': 10,
 'ph': 4.4,
 'attenuation_level': 75,
 'volume': {'value': 20, 'unit': 'litres'},
 'boil_volume': {'value': 25, 'unit': 'litres'},
 'method': {'mash_temp': [{'temp': {'value': 64, 'unit': 'celsius'},
    'duration': 75}],
  'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}},
  'twist': None},
 'ingredients': {'malt': [{'name': 'Maris Otter Extra Pale',
    'amount': {'value': 3.3, 'unit': 'kilograms'}},
   {'name': 'Caramalt', 'amount': {'value': 0.2, 'unit': 'kilograms'}},
   {'name': 'Munich', 'amount': {'value': 0.4, 'unit': 'kilograms'}}],
  'hops': [{'name': 'Fuggles',
    'amount': {'value': 25, 'unit': 'grams'},
    '

In [7]:
>>> next(beers)

{'id': 2,
 'name': 'Trashy Blonde',
 'tagline': "You Know You Shouldn't",
 'first_brewed': '04/2008',
 'description': 'A titillating, neurotic, peroxide punk of a Pale Ale. Combining attitude, style, substance, and a little bit of low self esteem for good measure; what would your mother say? The seductive lure of the sassy passion fruit hop proves too much to resist. All that is even before we get onto the fact that there are no additives, preservatives, pasteurization or strings attached. All wrapped up with the customary BrewDog bite and imaginative twist.',
 'image_url': 'https://images.punkapi.com/v2/2.png',
 'abv': 4.1,
 'ibu': 41.5,
 'target_fg': 1010,
 'target_og': 1041.7,
 'ebc': 15,
 'srm': 15,
 'ph': 4.4,
 'attenuation_level': 76,
 'volume': {'value': 20, 'unit': 'litres'},
 'boil_volume': {'value': 25, 'unit': 'litres'},
 'method': {'mash_temp': [{'temp': {'value': 69, 'unit': 'celsius'},
    'duration': None}],
  'fermentation': {'temp': {'value': 18, 'unit': 'celsius'}},
 

In [55]:
import psycopg2

con = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password='lol',
    port='5432',
)
con.autocommit = True

In [62]:
def copy_string_iterator(connection, beers: Iterator[Dict[str, Any]], size: int = 8192) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)
        beers_string_iterator = StringIteratorIO((
            '|'.join(map(clean_csv_value, (
                beer['id'],
                beer['name'],
                beer['tagline'],
                parse_first_brewed(beer['first_brewed']).isoformat(),
                beer['description'],
                beer['image_url'],
                beer['abv'],
                beer['ibu'],
                beer['target_fg'],
                beer['target_og'],
                beer['ebc'],
                beer['srm'],
                beer['ph'],
                beer['attenuation_level'],
                beer['brewers_tips'],
                beer['contributed_by'],
                beer['volume']['value'],
            ))) + '\n'
            for beer in beers
        ))
        cursor.copy_from(beers_string_iterator, 'beers', sep='|', size=size)

In [63]:
>>> with connection.cursor() as cursor:
>>>     create_staging_table(cursor)
beers = list(iter_beers_from_api())

In [64]:
len(beers)

325

In [65]:
import requests

def iter_beers_from_api(page_size: int = 25) -> Iterator[Dict[str, Any]]:
    session = requests.Session()
    page = 1
    while True:
        response = session.get('https://api.punkapi.com/v2/beers?' + urlencode({
            'page': page,
            'per_page': page_size
        }))
        response.raise_for_status()

        data = response.json()
        if not data:
            break

        for beer in data:
            yield beer

        page += 1


def iter_beers_from_file(path: str) -> Iterator[Dict[str, Any]]:
    import json
    with open(path, 'r') as f:
        data = json.load(f)
        for beer in data:
            yield beer


In [71]:
import io

def clean_csv_value(value:[Any]) -> str:
    if value is None:
        return r'\N'
    return str(value).replace('\n', '\\n')

def copy_stringio(connection, beers: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)
        csv_file_like_object = io.StringIO()
        for beer in beers:
            csv_file_like_object.write('|'.join(map(clean_csv_value, (
                beer['id'],
                beer['name'],
                beer['tagline'],
                parse_first_brewed(beer['first_brewed']),
                beer['description'],
                beer['image_url'],
                beer['abv'],
                beer['ibu'],
                beer['target_fg'],
                beer['target_og'],
                beer['ebc'],
                beer['srm'],
                beer['ph'],
                beer['attenuation_level'],
                beer['contributed_by'],
                beer['brewers_tips'],
                beer['volume']['value'],
            ))) + '\n')
        csv_file_like_object.seek(0)
        cursor.copy_from(csv_file_like_object, 'staging_beers', sep='|')
        
        con.commit() 

In [44]:
from datetime import datetime
from pgcopy import CopyManager
import psycopg2

cols = ('id', 'timestamp', 'location', 'temperature')
now = datetime.now()
records = [
        (0, now, 'Jerusalem', 72.2),
        (1, now, 'New York', 75.6),
        (2, now, 'Moscow', 54.3),
    ]
conn = psycopg2.connect(database='beer')
mgr = CopyManager(conn, 'measurements_table', cols)
mgr.copy(records)

# don't forget to commit!
conn.commit()

OperationalError: fe_sendauth: no password supplied


In [61]:
cur = con.cursor()  
cur.execute("SELECT admission, name, age, course, department from STUDENT")  
rows = cur.fetchall()

ProgrammingError: relation "student" does not exist
LINE 1: SELECT admission, name, age, course, department from STUDENT
                                                             ^
